In [26]:
import trimesh
import torch
import numpy as np
import imageio
assert(trimesh.ray.has_embree)


TypeError: unsupported operand type(s) for /: 'module' and 'int'

In [32]:
target_mesh = trimesh.load("/home/jiahui/DR/DR/data/untitled.obj")
# target_mesh.vertices.shape
target_mesh.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [1,0,0]))

<trimesh.Trimesh(vertices.shape=(298672, 3), faces.shape=(582206, 3))>

In [38]:
# scene will have automatically generated camera and lights
scene = target_mesh.scene()
# any of the automatically generated values can be overridden
# set resolution, in pixels
scene.camera.resolution = [100, 100]
# set field of view, in degrees
# make it relative to resolution so pixels per degree is same
scene.camera.fov = (120,120)
# convert the camera to rays with one ray per pixel
origins, vectors, pixels = scene.camera_rays()

In [39]:
scene = target_mesh.scene()
scene.camera_transform

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [-0.        ,  0.        ,  1.        ,  1.47058678],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [6]:
bunny = trimesh.load("../bunny.obj")
bunny.vertices -= bunny.center_mass
bunny.apply_transform([[1,0,0,0],
                        [0,0,1,0],
                        [0,-1,0,0],
                        [0,0,0,1]])
# bak_vertices = bunny.vertices
bak_vertices = torch.tensor(bunny.vertices,dtype=torch.float)


In [3]:
bunny_sm = trimesh.load("bunny_sm_sub.obj")
bunny_sm.vertices -= bunny_sm.center_mass
bunny_sm.apply_transform([[1,0,0,0],
                        [0,0,1,0],
                        [0,-1,0,0],
                        [0,0,0,1]])
_=bunny_sm.export("start.obj")

In [7]:
#simple camera
x = torch.linspace(-0.2, 0.2, 256)
z = torch.linspace(-0.2, 0.2, 256)
xv,zv=torch.meshgrid(x,z)
yv = torch.ones_like(xv)

ray_dir = torch.stack((xv.reshape(-1), yv.reshape(-1), zv.reshape(-1)), dim=1)
origin = torch.zeros_like(ray_dir) + torch.tensor([0,-0.5,0])

In [5]:
def render_norm(vertices, mesh):
    ind_tri, ind_ray = mesh.ray.intersects_id(origin, ray_dir,False, 1, False)
    ind_vert = mesh.faces[ind_tri]

    # <<Fast, Minimum Storage Ray/Triangle Intersection>> 
    # https://cadxfem.org/inf/Fast%20MinimumStorage%20RayTriangle%20Intersection.pdf
    def dot(v1, v2):
        return v1[:,0]*v2[:,0] + v1[:,1]*v2[:,1] + v1[:,2]*v2[:,2]

    v0 = vertices[ind_vert[:,0]]
    v1 = vertices[ind_vert[:,1]]
    v2 = vertices[ind_vert[:,2]]
    n0 = torch.tensor(mesh.vertex_normals[ind_vert[:,0]], dtype=torch.float)
    n1 = torch.tensor(mesh.vertex_normals[ind_vert[:,1]], dtype=torch.float)
    n2 = torch.tensor(mesh.vertex_normals[ind_vert[:,2]], dtype=torch.float)

    # Find vectors for two edges sharing v[0]
    edge1 = v1-v0
    edge2 = v2-v0
    pvec = torch.cross(ray_dir[ind_ray], edge2)

    # If determinant is near zero, ray lies in plane of triangle
    det = dot(edge1, pvec)
    inv_det = 1/det
    # Calculate distance from v[0] to ray origin
    tvec = origin[ind_ray] - v0
    # Calculate U parameter
    u = dot(tvec, pvec) * inv_det
    qvec = torch.cross(tvec, edge1)
    # Calculate V parameter
    v = dot(ray_dir[ind_ray], qvec) * inv_det
    # Calculate T
    t = dot(edge2, qvec) * inv_det
    # print( v.max(), v.min())
    assert( v.max()<=1.00001 and v.min()>=-0.00001)
    assert( u.max()<=1.00001 and u.min()>=-0.00001)
    assert( (v+u).max()<=1.00001 and (v+u).min()>=-0.00001)
    assert(t.min()>0)
    # interpolate normal
    n = (1-u-v).reshape((-1,1)) * n0 + u.reshape((-1,1)) * n1 + v.reshape((-1, 1)) * n2
    n = n / n.norm(dim=1).reshape(-1,1)

    img = torch.zeros(ray_dir.shape)
    img[ind_ray] = n
    img = img.reshape(256,256,3)

    return img

def render_facenorm(vertices, mesh):
    ind_tri, ind_ray = mesh.ray.intersects_id(origin, ray_dir,False, 1, False)
    ind_vert = mesh.faces[ind_tri]


    def dot(v1, v2):
        return v1[:,0]*v2[:,0] + v1[:,1]*v2[:,1] + v1[:,2]*v2[:,2]

    v0 = vertices[ind_vert[:,0]]
    v1 = vertices[ind_vert[:,1]]
    v2 = vertices[ind_vert[:,2]]

    # Find vectors for two edges sharing v[0]
    edge1 = v1-v0
    edge2 = v2-v0
    n = torch.cross(edge1, edge2)
    n = n / n.norm(dim=1).reshape(-1,1)

    img = torch.zeros(ray_dir.shape)
    img[ind_ray] = n
    img = img.reshape(256,256,3)

    return img

In [6]:
# render target normal map
bunny.vertices = bak_vertices
vertices = torch.tensor(bunny.vertices, dtype=torch.float)

# target = render_norm(vertices, bunny)
target = render_norm(vertices, bunny)
imageio.imsave("target.png", target)


Lossy conversion from float32 to uint8. Range [-0.9999842047691345, 0.999808669090271]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
for i in [-0.02, 0.0, 0.02]:
    for j in [-0.02, 0.0, 0.02]:
        offset = [i, 0, j]
        print("offset", offset, "gradient", cal_gradient(offset))


In [8]:
off = [0.02, 0, 0.02] # move right and down
parameter = torch.tensor(off, requires_grad=True)
opt = torch.optim.Adam([parameter], lr=.002)
gif = []
for it in range(30):
    # Zero out gradients before each iteration
    opt.zero_grad()
    vertices = bak_vertices + parameter.expand_as(bak_vertices)
    bunny.vertices = vertices.detach().numpy()
    img = render_norm(vertices, bunny)
    # imageio.imsave("img{}.png".format(off), img.detach())
    loss = (img-target).pow(2).sum()/(256*256)
    loss.backward()

    # Optimizer: take a gradient step
    opt.step()

    # imageio.imsave("img_{}.png".format(it), img.detach())
    print('Iteration %03i: error=%g' % (it, loss))

Iteration 000: error=0.365846
Iteration 001: error=0.351126
Iteration 002: error=0.337245
Iteration 003: error=0.320935
Iteration 004: error=0.297336
Iteration 005: error=0.262482
Iteration 006: error=0.215656
Iteration 007: error=0.159686
Iteration 008: error=0.0981118
Iteration 009: error=0.0463325
Iteration 010: error=0.0534497
Iteration 011: error=0.0726497
Iteration 012: error=0.0930754
Iteration 013: error=0.103609
Iteration 014: error=0.100506
Iteration 015: error=0.0879809
Iteration 016: error=0.0689747
Iteration 017: error=0.0482964
Iteration 018: error=0.0355741
Iteration 019: error=0.0347945
Iteration 020: error=0.04499
Iteration 021: error=0.0577449
Iteration 022: error=0.0645674
Iteration 023: error=0.0635588
Iteration 024: error=0.056293
Iteration 025: error=0.0437389
Iteration 026: error=0.029128
Iteration 027: error=0.0196685
Iteration 028: error=0.0203344
Iteration 029: error=0.0234401


In [ ]:
bunny_sm = trimesh.load("bunny_sm_sub.obj")
bunny_sm.vertices -= bunny_sm.center_mass
bunny_sm.apply_transform([[1,0,0,0],
                        [0,0,1,0],
                        [0,-1,0,0],
                        [0,0,0,1]])
sm_vertices = torch.tensor(bunny_sm.vertices,dtype=torch.float)
sm_normals = torch.tensor(bunny_sm.vertex_normals, dtype=torch.float)

# parameter = torch.zeros((bunny_sm.vertices.shape[0],1), requires_grad=True)
parameter = torch.zeros(bunny_sm.vertices.shape, requires_grad=True)
opt = torch.optim.Adam([parameter], lr=.00002)
gif = []
for it in range(90):
    # Zero out gradients before each iteration
    opt.zero_grad()

    # vertices = sm_vertices + parameter * sm_normals
    vertices = sm_vertices + parameter
    bunny_sm.vertices = vertices.detach().numpy()
    img = render_facenorm(vertices, bunny_sm)
    gif.append(img.detach().numpy())
    loss = (img-target).pow(2).sum()/(256*256)
    (loss + 0.1*parameter.pow(2).sum()).backward()

    # Optimizer: take a gradient step
    opt.step()

    # imageio.imsave("result/img_{}.png".format(it), img.detach())
    print('Iteration %03i: error=%g' % (it, loss))

In [ ]:
# sm_vertices = torch.tensor(bunny_sm.vertices,dtype=torch.float)

# parameter = torch.zeros((bunny_sm.vertices.shape[0],1), requires_grad=True)
parameter = torch.zeros(bunny_sm.vertices.shape, requires_grad=True)
opt = torch.optim.Adam([parameter], lr=.00005)
for it in range(90):
    # Zero out gradients before each iteration
    opt.zero_grad()

    # vertices = sm_vertices + parameter * sm_normals
    vertices = sm_vertices + parameter
    bunny_sm.vertices = vertices.detach().numpy()
    img = render_facenorm(vertices, bunny_sm)
    loss = (img-target).pow(2).sum()/(256*256)
    (loss + 0.1*parameter.pow(2).sum()).backward()

    # Optimizer: take a gradient step
    opt.step()

    # imageio.imsave("result/img_{}.png".format(it), img.detach())
    print('Iteration %03i: error=%g' % (it, loss))

In [9]:
imageio.mimsave("optim.gif", gif)

RuntimeError: Zero images were written.